We can repeat the steps from Regression Project..We have to  rename the project directory name and the git remote configuration should be remote

In [1]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /Users/loonycorn/projects/dvc/churn_prediction/.git/


In [2]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [1]:
!git config --global user.name "loonytest"  
!git config --global user.email "loony.test.001@gmail.com"

!git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	.ipynb_checkpoints/
	data/
	demo-03-ChurnPrediction.ipynb
	dvclive/
	model_dir/

nothing added to commit but untracked files present (use "git add" to track)


In [2]:
!git remote add origin https://github.com/loonybq/dvc_churn_prediction.git
    
!git branch -M main

!git push -u origin main

error: remote origin already exists.
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 438 bytes | 438.00 KiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/loonybq/dvc_churn_prediction.git
   02d02ff..2d667cb  main -> main
branch 'main' set up to track 'origin/main'.


In [3]:
!mkdir /tmp/dvc_storage_cp

!dvc remote add -d localremote /tmp/dvc_storage_cp

!dvc remote list

mkdir: /tmp/dvc_storage_cp: File exists
Setting 'localremote' as a default remote.
ERROR: configuration error - config file error: remote 'localremote' already exists. Use `-f|--force` to overwrite it.
localremote	/tmp/dvc_storage_cp


In [18]:
!git add .dvc/config

In [19]:
!git commit -m "Initialized DVC for churn prediction"

[main 2d667cb] Initialized DVC for churn prediction
 1 file changed, 4 insertions(+)


In [16]:
!git push -u origin main

branch 'main' set up to track 'origin/main'.
Everything up-to-date


# TODO Recording:

- Go to GitHub repo and refresh and show that the contents have been pushed
- Click on the project and show that there is one commit visible
- Open Iterative Studio on a new tab
- Add a new project connected to this repo
- Click on the Account to the top right -> Profile
- Scroll to the bottom and show that we can generate a new token
- We don't need to generate a new token, we will use the same one as before
- Come back to this notebook and continue


In [4]:
!dvc config --global studio.token isat_3Ak630M1uyW8zHqg0vyhjJH9vLv0i7Ug1TxTgzxAGTEhmph8m

Installing packages required

In [5]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [6]:
travel_churn_data = pd.read_csv('data/CustomerTravel.csv')

travel_churn_data.head()

Age FrequentFlyer AnnualIncomeClass  ServicesOpted  \
0   34            No     Middle Income              6   
1   34           Yes        Low Income              5   
2   37            No     Middle Income              3   
3   30            No     Middle Income              2   
4   30            No        Low Income              1   

  AccountSyncedToSocialMedia BookedHotelOrNot  Target  
0                         No              Yes       0  
1                        Yes               No       1  
2                        Yes               No       0  
3                         No               No       0  
4                         No               No       0

In [7]:
travel_churn_data.shape

(954, 7)

In [8]:
travel_churn_data.columns

Index(['Age', 'FrequentFlyer', 'AnnualIncomeClass', 'ServicesOpted',
       'AccountSyncedToSocialMedia', 'BookedHotelOrNot', 'Target'],
      dtype='object')

In [9]:
travel_churn_data = travel_churn_data.drop_duplicates()
travel_churn_data.shape

(447, 7)

In [10]:
travel_churn_data['Target'].value_counts()

Target
0    325
1    122
Name: count, dtype: int64

In [11]:
travel_churn_data['AnnualIncomeClass'].unique()

array(['Middle Income', 'Low Income', 'High Income'], dtype=object)

In [12]:
mapper = {'Low Income': 0,'Middle Income': 1, 'High Income': 2}

travel_churn_data['AnnualIncomeClass'] = travel_churn_data['AnnualIncomeClass'].replace(mapper)

In [13]:
categorical_features = [
    'FrequentFlyer',
    'AccountSyncedToSocialMedia', 
    'BookedHotelOrNot'
]
categorical_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))]
)

preprocessor = ColumnTransformer(
    transformers = [('cat_tr', categorical_transformer, categorical_features)], 
    remainder = StandardScaler()
)

X = travel_churn_data.drop(labels = ['Target'], axis = 1)
 
y = travel_churn_data['Target']
 
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 124)

Logistic Regression Model is trained and parameters and metrics are logged.
We can display the display the logged parameters and metrics inside the notebook using report='notebook'.
We can generate markdown reports also using report='md'. First of all, we will run LR model with default parameters. Then we will set the class_weight parameter to balanced.When we do metrics comparison, Recall score is improved for 'balanced' LR model

### Notes:

- Note that we're only logging params and metrics (we'll log artifacts for other models that perform well)
- We'll use the same dvclive folder for all out models (so each model is not stored in a different folder)

# TODO Recording

- report = 'notebook' will generate the report within this notebook, if that does not work in your test run please just remove the parameter

In [14]:
from dvclive import Live

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [13]:
from dvclive import Live

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

lr_model = LogisticRegression()
    
pipe_lr = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', lr_model)])

model_params = lr_model.get_params()
print(model_params)
pipe_lr.fit(X_train, y_train)
 
predictions = pipe_lr.predict(X_test)
predictions_predict_prob = pipe_lr.predict_proba(X_test)
    
train_accuracy_score = pipe_lr.score(X_train, y_train)
test_accuracy_score = accuracy_score(y_test, predictions)
test_precision_score = precision_score(y_test, predictions)
test_recall_score = recall_score(y_test, predictions)
test_f1_score = f1_score(y_test, predictions)
auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])
    
metrics = {
    'Train_accuracy_score': train_accuracy_score, 
    'Test_accuracy_score': test_accuracy_score,
    'Test_precision_score': test_precision_score,
    'Test_recall_score': test_recall_score,
    'Test_f1_score': test_f1_score,
    'AUC_score': auc_score
}
    
with Live(
    save_dvc_exp = True, 
    report = 'notebook', 
    exp_message = 'Logistic regression churn classification'
) as live:
    live.log_params(model_params)
        
    live.log_metric('Train_accuracy_score',train_accuracy_score)
    live.log_metric('Test_accuracy_score',test_accuracy_score)
    live.log_metric('Test_precision_score', test_precision_score)
    live.log_metric('Test_recall_score', test_recall_score)
    live.log_metric('Test_f1_score', test_f1_score)
    live.log_metric('AUC_score', auc_score)
    
    live.log_sklearn_plot('confusion_matrix', y_test, predictions, name = 'cm.json')

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}



<div style="width: 100%;height: 700px;text-align: center">
DVCLive Report
</div>


	.DS_Store, demo-03-ChurnPrediction.ipynb, .ipynb_checkpoints/demo-03-ChurnPrediction-checkpoint.ipynb, data/.DS_Store, data/.gitignore


# TODO Recording

- Go to the the Finder window for ~/projects/dvc/dvc_churn_prediction, show that we have a dvclive/ folder
- Drag that folder into Sublimetext and show the dvc.yaml, metrics.json, and params.yaml file
- Show the cm.json under sklearn plot
- Switch to a terminal window in ~/projects/dvc/dvc_churn_prediction
- Run the following commands

`dvc exp list`

`dvc exp show`

- Hit :q to quit

- Copy the experiment id and run in the terminal

`dvc exp push origin <experiment_id>`

- Go to Studio and show the metrics - this is not a very good model
- Show the plots as well, the confusion matrix is a new plot
- Hover over the sections of the confusion metrics to show


Let's improve this model by using class_weight = 'balanced'

In [14]:
lr_model = LogisticRegression(class_weight = 'balanced')
    
pipe_lr = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', lr_model)])

model_params = lr_model.get_params()
print(model_params)
pipe_lr.fit(X_train, y_train)
 
predictions = pipe_lr.predict(X_test)
predictions_predict_prob = pipe_lr.predict_proba(X_test)
    
train_accuracy_score = pipe_lr.score(X_train, y_train)
test_accuracy_score = accuracy_score(y_test, predictions)
test_precision_score = precision_score(y_test, predictions)
test_recall_score = recall_score(y_test, predictions)
test_f1_score = f1_score(y_test, predictions)
auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])
    
metrics = {
    'Train_accuracy_score': train_accuracy_score, 
    'Test_accuracy_score': test_accuracy_score,
    'Test_precision_score': test_precision_score,
    'Test_recall_score': test_recall_score,
    'Test_f1_score': test_f1_score,
    'AUC_score': auc_score
}
    
with Live(
    save_dvc_exp = True, 
    report = 'notebook', 
    exp_message = 'Logistic regression balanced weight churn classification'
) as live:
    live.log_params(model_params)
        
    live.log_metric('Train_accuracy_score',train_accuracy_score)
    live.log_metric('Test_accuracy_score',test_accuracy_score)
    live.log_metric('Test_precision_score', test_precision_score)
    live.log_metric('Test_recall_score', test_recall_score)
    live.log_metric('Test_f1_score', test_f1_score)
    live.log_metric('AUC_score', auc_score)
    
    live.log_sklearn_plot('confusion_matrix', y_test, predictions, name = 'cm.json')
    

{'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}



<div style="width: 100%;height: 700px;text-align: center">
DVCLive Report
</div>


	.DS_Store, demo-03-ChurnPrediction.ipynb, .ipynb_checkpoints/demo-03-ChurnPrediction-checkpoint.ipynb, data/.DS_Store, data/.gitignore


# TODO Recording

- Switch to a terminal window in ~/projects/dvc/dvc_churn_prediction
- Run the following commands

`dvc exp list`

`dvc exp show`

- We now have two experiments
- Copy the experiment id for the new experiment and run in the terminal

`dvc exp push origin <experiment_id>`

- Go to Studio and show the metrics - the recall and F1 scores of the model have improved
- Select both experiments and compare and show the improvement

RandomForest Model is trained and parameters and metrics are logged.It can easily seen that this experiment's performance is quite better than Logistic Regression Experiment.


In [15]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
    
pipe_rf = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', rf_model)])

model_params = rf_model.get_params()
print(model_params)
pipe_rf.fit(X_train, y_train)

predictions = pipe_rf.predict(X_test)
predictions_predict_prob = pipe_rf.predict_proba(X_test)
    
train_accuracy_score = pipe_rf.score(X_train, y_train)
test_accuracy_score = accuracy_score(y_test, predictions)
test_precision_score = precision_score(y_test, predictions)
test_recall_score = recall_score(y_test, predictions)
test_f1_score = f1_score(y_test, predictions)
auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])
    
metrics = {
    'Train_accuracy_score': train_accuracy_score, 
    'Test_accuracy_score': test_accuracy_score,
    'Test_precision_score': test_precision_score,
    'Test_recall_score': test_recall_score,
    'Test_f1_score': test_f1_score,
    'AUC_score': auc_score
}
    
with Live(
    save_dvc_exp = True, 
    exp_message = 'Random forest churn classification'
) as live:
    live.log_params(model_params)
        
    live.log_metric('Train_accuracy_score',train_accuracy_score)
    live.log_metric('Test_accuracy_score',test_accuracy_score)
    live.log_metric('Test_precision_score', test_precision_score)
    live.log_metric('Test_recall_score', test_recall_score)
    live.log_metric('Test_f1_score', test_f1_score)
    live.log_metric('AUC_score', auc_score)
    
    live.log_sklearn_plot('confusion_matrix', y_test, predictions, name = 'cm.json')

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


	.DS_Store, demo-03-ChurnPrediction.ipynb, .ipynb_checkpoints/demo-03-ChurnPrediction-checkpoint.ipynb, data/.DS_Store, data/.gitignore


# TODO Recording

- Switch to a terminal window in ~/projects/dvc/dvc_churn_prediction

`dvc exp show`

- Show that the metrics for the RF model is a huge improvement
- We'll push this to Studio later

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(class_weight = 'balanced')
    
pipe_rf = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', rf_model)])

model_params = rf_model.get_params()
print(model_params)
pipe_rf.fit(X_train, y_train)

predictions = pipe_rf.predict(X_test)
predictions_predict_prob = pipe_rf.predict_proba(X_test)
    
train_accuracy_score = pipe_rf.score(X_train, y_train)
test_accuracy_score = accuracy_score(y_test, predictions)
test_precision_score = precision_score(y_test, predictions)
test_recall_score = recall_score(y_test, predictions)
test_f1_score = f1_score(y_test, predictions)
auc_score = roc_auc_score(y_test,  predictions_prebict_prob[:, 1])
    
metrics = {
    'Train_accuracy_score': train_accuracy_score, 
    'Test_accuracy_score': test_accuracy_score,
    'Test_precision_score': test_precision_score,
    'Test_recall_score': test_recall_score,
    'Test_f1_score': test_f1_score,
    'AUC_score': auc_score
}
    
with Live(
    save_dvc_exp = True, 
    exp_message = 'Random forest balanced weight churn classification'
) as live:
    live.log_params(model_params)
        
    live.log_metric('Train_accuracy_score',train_accuracy_score)
    live.log_metric('Test_accuracy_score',test_accuracy_score)
    live.log_metric('Test_precision_score', test_precision_score)
    live.log_metric('Test_recall_score', test_recall_score)
    live.log_metric('Test_f1_score', test_f1_score)
    live.log_metric('AUC_score', auc_score)
    
    live.log_sklearn_plot('confusion_matrix', y_test, predictions, name = 'cm.json')

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


	.DS_Store, demo-03-ChurnPrediction.ipynb, .ipynb_checkpoints/demo-03-ChurnPrediction-checkpoint.ipynb, data/.DS_Store, data/.gitignore


# TODO Recording

- Switch to a terminal window in ~/projects/dvc/dvc_churn_prediction
- Run the following commands

`dvc exp show`

- Copy the experiment id for the two RF experiments and dvc push them

`dvc exp push origin <experiment_id>`

`dvc exp push origin <experiment_id>`

- Go to Studio and show the metrics 
- Select the RF experiments and compare and show how they are different (only the differences will be shown)

In [17]:
pip install xgboost

     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.8 MB 3.1 MB/s eta 0:00:01━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.8 MB 4.4 MB/s eta 0:00:01━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/1.8 MB 5.5 MB/s eta 0:00:01━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 0.7/1.8 MB 3.8 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 1.1/1.8 MB 4.0 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 MB 5.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In the context of the XGBClassifier from the XGBoost library, the **scale_pos_weight parameter** is used to address class imbalance in binary classification problems. It is an important hyperparameter that helps to balance the effect of the positive class (class 1) against the negative class (class 0) during the training of the XGBoost model.

The scale_pos_weight parameter helps address this imbalance by assigning a higher weight to the positive class during the training process. When set to a value greater than 1, it indicates that the positive class is underrepresented relative to the negative class. The actual weight assigned to the positive class is calculated as:

scale_pos_weight = (number of negative samples) / (number of positive samples)

For example, setting scale_pos_weight=2 means that the positive class is about half the size of the negative class, and XGBoost will assign a higher weight to the positive class during training to balance the impact of the two classes.

In [15]:
import os
from joblib import dump

from xgboost.sklearn import XGBClassifier

xgb_model = XGBClassifier(scale_pos_weight=2)
    
pipe_xgb = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', xgb_model)])
pipe_xgb.fit(X_train, y_train)

model_params = xgb_model.get_params()
print(model_params)

# Create a path where we store the serialized model
os.makedirs('model_dir', exist_ok = True)
model_path = os.path.join('model_dir', "churn-prediction.joblib")

# Store the model in this directory
dump(pipe_xgb, model_path)

predictions =  pipe_xgb.predict(X_test)
predictions_predict_prob = pipe_xgb.predict_proba(X_test)

train_accuracy_score = pipe_xgb.score(X_train, y_train)
test_accuracy_score = accuracy_score(y_test, predictions)
test_precision_score = precision_score(y_test, predictions)
test_recall_score = recall_score(y_test, predictions)
test_f1_score = f1_score(y_test, predictions)
auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])
    
metrics = {
    'Train_accuracy_score': train_accuracy_score, 
    'Test_accuracy_score': test_accuracy_score,
    'Test_precision_score': test_precision_score,
    'Test_recall_score': test_recall_score,
    'Test_f1_score': test_f1_score,
    'AUC_score': auc_score
}
    
with Live(
    save_dvc_exp = True, 
    exp_message = 'XGBoost with balanced weight churn classification'
) as live:
    live.log_params(model_params)
 
    live.log_artifact(
        "data/Customertravel.csv",
        type="dataset",
        name="customertravel",
        desc="Customer travel churn prediction dataset",
        labels=["classification", "churn"],
    )
    
    live.log_metric('Train_accuracy_score', train_accuracy_score)
    live.log_metric('Test_accuracy_score', test_accuracy_score)
    live.log_metric('Test_precision_score', test_precision_score)
    live.log_metric('Test_recall_score', test_recall_score)
    live.log_metric('Test_f1_score', test_f1_score)
    live.log_metric('AUC_score', auc_score)
    
    live.log_sklearn_plot('confusion_matrix', y_test, predictions, name = 'cm.json')
    
    live.log_artifact(model_path, type = 'model')

{'objective': 'binary:logistic', 'use_label_encoder': None, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': 2, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


	.DS_Store, demo-03-ChurnPrediction.ipynb, .ipynb_checkpoints/demo-03-ChurnPrediction-checkpoint.ipynb, data/.DS_Store


# TODO Recording

- Switch to a terminal window in ~/projects/dvc/dvc_churn_prediction


- Run the following commands
- Show the artifacts registered

`cat dvclive/dvc.yaml`

`dvc exp show`

- Copy the experiment id for the XGBoost experiment and dvc push them

`dvc exp push origin <experiment_id>`

- Now let's commit all changes related to the experiment
- Head over to Studio and show the latest experiment and its metrics - it's the best performing model so far

##### Create a git branch

- Click on dvc_churn_prediction on top and go to the main Project page for this repo
- Click on the 3 dots next to the experiment and select the option "Create branch/pull request"
- Use the default branch name 
- Select the checkbox "Create a pull request after creating a branch?"
- Choose base branch as main
- Create branch
- Go to this repository on GitHub
- Notice that we now have two branches! We have pushed the changes related to this experiment to a different branch

##### Explore branch and merge with main
- Select the experiment_id-branch
- Click on the files in the branch and show a few important files (use the left navigation pane)
- dvc.yaml, metrics.json, params.yaml, data/customertravel.csv.dvc, model_dir/churn-prediction.joblib.dvc files
- Show that on top the "Pull Requests" tab shows a notification
- Click on that tab and see that the request is about merging the experiment_id-branch with main
- Click on merge and perform the merge 
- Now go to the main branch and all the experiment files should be here

##### Model registration
- Go to the models page (the model will be logged and available there)
- Register the model as v1.0.0
- Click on the 3 dots next to the mode and go to "View model page"
- Copy over the dvc get command to get the model file


##### Accessing model for predictions

- Back to the terminal window

`mkdir loaded_model`

`cd loaded_model`

`dvc get https://github.com/loonybq/dvc_churn_prediction ../model_dir/churn-prediction.joblib --rev 4ec1f975cd27f9b0a74b49012d5438ea00ba560f`

- Show the model has been downloaded
`ls -l`

- Come back to this notebook

In [17]:
from joblib import load

loaded_model = load('loaded_model/churn-prediction.joblib')

In [18]:
predictions_loaded = loaded_model.predict(X_test)
 
predictions_loaded

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1])

In [20]:
predictions_original = predictions
 
predictions_original

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1])

In [21]:
assert(np.array_equal(predictions_loaded, predictions_original))

#### Python API

In [24]:
import dvc.api

exps = dvc.api.exp_show()

exps

[{'Experiment': None,
  'rev': 'workspace',
  'typ': 'baseline',
  'Created': None,
  'parent': None,
  'State': None,
  'Executor': None,
  'Train_accuracy_score': 0.9607843137254902,
  'Test_accuracy_score': 0.7777777777777778,
  'Test_precision_score': 0.5625,
  'Test_recall_score': 0.75,
  'Test_f1_score': 0.6428571428571429,
  'AUC_score': 0.8693181818181819,
  'objective': 'binary:logistic',
  'use_label_encoder': 'None',
  'base_score': 'None',
  'booster': 'None',
  'callbacks': 'None',
  'colsample_bylevel': 'None',
  'colsample_bynode': 'None',
  'colsample_bytree': 'None',
  'early_stopping_rounds': 'None',
  'enable_categorical': 'False',
  'eval_metric': 'None',
  'feature_types': 'None',
  'gamma': 'None',
  'gpu_id': 'None',
  'grow_policy': 'None',
  'importance_type': 'None',
  'interaction_constraints': 'None',
  'learning_rate': 'None',
  'max_bin': 'None',
  'max_cat_threshold': 'None',
  'max_cat_to_onehot': 'None',
  'max_delta_step': 'None',
  'max_depth': 'None'

Gets the latest metrics and params from the main branch of the Git repository. This will be our XGBoost model

In [26]:
metrics = dvc.api.metrics_show()

metrics

{'Train_accuracy_score': 0.9607843137254902,
 'Test_accuracy_score': 0.7777777777777778,
 'Test_precision_score': 0.5625,
 'Test_recall_score': 0.75,
 'Test_f1_score': 0.6428571428571429,
 'AUC_score': 0.8693181818181819}

In [28]:
params = dvc.api.params_show()

params

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': 2,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}